# __3. 통계적 실험과 유의성 검정__

##### 파이썬 패키지, 데이터 import

In [2]:
%matplotlib inline

from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.utils import resample

import seaborn as sns
import matplotlib.pylab as plt

In [3]:
state = pd.read_csv('./data/state.csv')
dfw = pd.read_csv('./data/dfw_airline.csv')
sp500_px = pd.read_csv('./data/sp500_data.csv.gz', index_col=0)
sp500_sym = pd.read_csv('./data/sp500_sectors.csv')
kc_tax = pd.read_csv('./data/kc_tax.csv.gz')
lc_loans = pd.read_csv('./data/lc_loans.csv')
airline_stats = pd.read_csv('./data/airline_stats.csv')
loans_income = pd.read_csv('./data/loans_income.csv').squeeze('columns')

#### 통계적 추론 : 제한된 데이터로 주어진 실험 결과를 더 큰 과정 또는 모집단에 적용하려는 의도를 반영한다.
#### 통계 추론 과정
1. 가설을 세운다
2. 실험을 설계한다.
3. 데이터를 수집한다.
4. 추론 및 결론을 도출한다.

## __3.1 A/B 검정__

#### __A/B 검정__ : 두 가지 처리 방법 중 어느 쪽이 다른 쪽보다 우월하다는 것을 알기 위해 __실험군을 두 그룹으로 나누어__ 진행하는 실험

#### __처리(treatment)__ : 어떤 대상에 주어지는 특별한 환경이나 조건
#### 서로 다른 처리 조건을 제외한 __나머지 조건들은 정확히 동일하게 처리된다.__
#### __처리군(처리 그룹)__ : 특정 처리에 노출된 대상들의 집단
#### __대조군(대조 그룹, control group)__ : 어떤 처리도 하지 않은 대상들의 집단

#### __임의화(ramdomization)__ : 처리를 적용할 대상을 임의로 결정하는 과정
#### __대상(subject)__ : 처리를 적용할 개체 대상
#### __검정통계량(test statistic)__ : 처리 효과를 측정하기 위한 지표

#### 처리군 간의 차이는 다음의 이유로 발생함
* 다른 처리의 효과
* 어떤 대상이 어떤 처리에 배정될 지에 대한 경우의 수(무작위로 배정한 결과, 자연스럽게 더 좋은 결과를 보이는 대상들이 한 쪽에 집중됨)

#### __대조군이 필요한 이유__ : 대조군이 없을 경우, '모든 다른 것들은 동일하다'는 보장이 없으며 어떤 차이가 처리 혹은 우연 때문인 지 알 수 없다.

#### 일반적인 A/B 검정 실험에서는 __미리 하나의 측정 지표를 결정해야 한다.__
#### 실험을 수행한 뒤 나중에 검정통계량을 선택한다면, __연구자 편향__ 이라는 함정에 빠지게 된다.

## __3.2 가설검정__

#### __가설검정(hypothesis test)__ : 관찰된 효과가 우연에 의한 것인지 여부를 나타내는 것 __(= 유의성 검정, significance test)__
#### 연구자가 랜덤하게 우연히 일어난 일에 속지 않도록 보호하기 위한 방법

#### ex: 동전던지기 50번 수행한다고 가정, 앞면(H)과 뒷면(T)을 예측해서 기록 후 실제로 50회 던지기 수행
#### 실제로 동전을 던진 결과를 임의로 예측한 결과와 구분하여 서로 다른 더미에 놓도록 함
#### 실제 결과에는 H or T가 연속적으로 나오는 경우가 더 길게 나타남.
#### 그러나, 대부분으 사람에게 랜덤한 동전 던지기를 예측하라고 했을 때, H를 3~4번 연속으로 나오게 한 뒤에ㅐ는 무작위이기 위해서 T가 나올 차례라고 속임.

#### 적절하게 설계된 A/B 검정에서, A와 B 사이의 관찰된 차이가 다음의 원인들로 설명될 수 있도록 A와 B에 대한 데이터를 수집
* 우연한 대상 선정
* A와 B의 진정한 차이

#### 통계 가설검정은 그룹 A와 B 사이에서 보이는 차이가 우연에 의한 것인지 평가하기 위해, 더 나아가 그 외 여러 무작위 실험을 포함하는 분석

### __3.2.1 귀무가설__

#### 가설검정의 전제 : 실험에서 얻은 그룹 간의 차이가 무작위로 얻을 수 있는 __합리적인 수준과는 극단적으로 다르다는 증거__ 가 있다.

#### __귀무가설(null hypothesis)__ : __'그룹들이 보이는 결과는 서로 동일하며, 그룹 간의 차이는 우연에 결과이다'__ 라는 기본 가정
#### 귀무가설이 __틀렸다__ 라는 것을 입중하여, A그룹과 B그룹 간 차이가 우연이 아니라는 것을 보여주는 것이 목표

### __3.2.2 대립가설__

#### __대립가설(alternative hypothesis)__ : 귀무가설과 대조되는 가설, (증명하고자 하는 가설)

#### 대립가설과 귀무가설의 예시

1. * 귀무가설: 그룹 A와 그룹 B의 평균에는 차이가 없다
    * 대립가설: A와 B의 평균에는 차이가 있다.
2. * 귀무가설: A $\leq$ B
    * 대립가설: A > B
3. * 귀무가설: B는 A보다 X% 더 크지 않다.
    * 대립가설: B는 A보다 X% 크다.

### __3.2.3 일원/이원 가설검정__

#### __일원 검정(one-way test)__ : 한 방향(단방향)으로만 우연히 일어날 확률을 계산하는 가설검정(한 쪽 꼬리)
#### __이원 검정(two-way test)__ : 양방향으로 우연히ㅔ 일어날 확률을 계산하는 가설검정 (양쪽 꼬리) -> default